In [25]:
import numpy as np

import torch
import csv

from sklearn.manifold import TSNE
from sklearn.decomposition import PCA, TruncatedSVD
from tqdm import tqdm

from lung_function.modules.networks import get_net_3d
from lung_function.modules.set_args import get_args
from lung_function.modules.path import PFTPath
from lung_function.modules.datasets import all_loaders
from lung_function.modules import provider

import sys
sys.path.append("../../modules/networks/models_pcd")


# Generate the high dimensional features and save them to a file.

In [15]:
args = get_args(jupyter=True)

# specify the run id and fold
args.id = 1655
args.fold = 1
args.pointnet_fc_ls = '1024-128'
mypath = PFTPath(args.id, check_id_dir=False, space=args.ct_sp)
device = torch.device("cuda:0")


In [16]:
# load network
target = [i.lstrip() for i in args.target.split('-')]
pointnet_fc_ls =  [int(i) for i in args.pointnet_fc_ls.split('-')]
net = get_net_3d(name=args.net, nb_cls=len(target), image_size=args.x_size,
                              pretrained=args.pretrained_imgnet, pointnet_fc_ls=pointnet_fc_ls, loss=args.loss,
                              dp_fc1_flag = args.dp_fc1_flag)  # output FVC and FEV1

# load weights

ckpt = torch.load(mypath.model_fpath, map_location=device)

if type(ckpt) is dict and 'model' in ckpt:
    model = ckpt['model']
else:
    model = ckpt
# model_fpath need to exist
net.load_state_dict(model, strict=False)
# move the new initialized layers to GPU
net = net.to(device)

In [10]:
# load dataset
data_dt = all_loaders(mypath.data_dir, mypath.label_fpath, args)

-----
SSc_patient_0068800
SSc_patient_0152440
SSc_patient_0163750
SSc_patient_0213786
SSc_patient_0219312
SSc_patient_0222357
SSc_patient_0234084
SSc_patient_0242508
SSc_patient_0277377
SSc_patient_0280727
SSc_patient_0298877
SSc_patient_0315457
SSc_patient_0315573
SSc_patient_0343115
SSc_patient_0376540
SSc_patient_0385738
SSc_patient_0456204
SSc_patient_0509944
SSc_patient_0534778
SSc_patient_0553478
SSc_patient_0573200
SSc_patient_0611832
SSc_patient_0639432
SSc_patient_0703966
SSc_patient_0787506
SSc_patient_0911478
SSc_patient_0992750
SSc_patient_1043946
SSc_patient_1066422
SSc_patient_1100289
SSc_patient_1105441
SSc_patient_1146160
SSc_patient_1160750
SSc_patient_1199412
SSc_patient_1236033
SSc_patient_1239567
SSc_patient_1239678
SSc_patient_1278925
SSc_patient_1280678
SSc_patient_1293386
SSc_patient_1301116
SSc_patient_1321066
SSc_patient_1326574
SSc_patient_1329481
SSc_patient_1329821
SSc_patient_1333231
SSc_patient_1348417
SSc_patient_1391017
SSc_patient_1397732
SSc_patient_14

Loading dataset: 100%|██████████| 62/62 [00:09<00:00,  6.52it/s]


In [26]:
# Obtain features
net.eval()
feature_dt = {'train', [], 'valid', [], 'test', []}
for mode in ['train', 'valid', 'test']:
    dataloader = data_dt[mode]
    for data in tqdm(dataloader):
        if args.input_mode == 'vessel':
            key = 'vessel'
        elif args.input_mode == 'vessel_skeleton_pcd':
            key = 'vessel_skeleton_pcd'
            points = data[key].data.numpy()
            if points.shape[0] == 1:  # batch size=1
                points = np.concatenate([points, points])
                data['label'] = np.concatenate(
                    [data['label'], data['label']])
                data['label'] = torch.tensor(data['label'])

            points = provider.random_point_dropout(points)
            points[:, :, 0:3] = provider.random_scale_point_cloud(
                points[:, :, 0:3])
            points[:, :, 0:3] = provider.shift_point_cloud(
                points[:, :, 0:3])
            points = torch.Tensor(points)
            points = points.transpose(2, 1)
            data[key] = points
        elif 'ct_masked_by_vessel' in args.input_mode:
            key = args.input_mode

        else:
            key = 'image'
        batch_x = data[key].to(device)
        batch_y = data['label'].to(device)

        with torch.cuda.amp.autocast():
            with torch.no_grad():
                x, trans, trans_feat = net.feat(batch_x)



100%|██████████| 4/4 [00:00<00:00, 15.68it/s]


In [27]:
x.shape

torch.Size([2, 1024])

In [ ]:
# save features

head = ['pat_id']
head.extend(self.target)

batch_pat_id = data['pat_id'].cpu(
).detach().numpy()  # shape (N,1)
batch_pat_id = int2str(batch_pat_id)  # shape (N,1)

batch_y_np = batch_y.cpu().detach().numpy()  # shape (N, out_nb)
pred_np = pred.cpu().detach().numpy()  # shape (N, out_nb)
# batch_pat_id = np.expand_dims(batch_pat_id, axis=-1)  # change the shape from (N,) to (N, 1)

# shape (1,1)
if args.input_mode == 'vessel_skeleton_pcd' and len(batch_pat_id) == 1:
    batch_pat_id = np.array(
        [[int(batch_pat_id[0])], [int(batch_pat_id[0])]])
    batch_pat_id = torch.tensor(batch_pat_id)

saved_label = np.hstack((batch_pat_id, batch_y_np))
saved_pred = np.hstack((batch_pat_id, pred_np))
medutils.appendrows_to(self.mypath.save_label_fpath(
    mode), saved_label, head=head)
medutils.appendrows_to(
    self.mypath.save_pred_fpath(mode), saved_pred, head=head)

# Load the high dimensional data and do dimensionality reduction using PCA and TSNE

In [ ]:
# load data
data_fpath = 

In [ ]:

# high_d 为 numpy 格式的高维数据
# TSNE 方法只能降维数据至 3 维（包括）以下，专注可视化效果
low_d = TSNE(random_state=0).fit_transform(high_d) 

# PCA、SVD 方法可以降维数据到高于 3 维
low_d = PCA(random_state=0, n_components=128).fit_transform(high_d) 
low_d = TruncatedSVD(random_state=0, n_components=128).fit_transform(high_d) 
